<a href="https://colab.research.google.com/github/novus-afk/self_eda_imdb/blob/master/EDA_on_IMDB_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## You have been hired by a rookie movie producer to help him decide what type of movies to produce and which actors to cast. You have to back your recommendations based on thorough analysis of the data he shared with you which has the list of 3000 movies and the corresponding details.

## As a data scientist, you have to first explore the data and check its sanity.

## Further, you have to answer the following questions:
1. ### <b> Which movie made the highest profit? Who were its producer and director? Identify the actors in that film.</b>
2. ### <b>This data has information about movies made in different languages. Which language has the highest average ROI (return on investment)? </b>
3. ### <b> Find out the unique genres of movies in this dataset.</b>
4. ### <b> Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI? </b>
5. ### <b> Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor. </b>
6. ### <b>Top 3 directors prefer which actors the most? </b>



# Data Exploration

In [ ]:
#Import package
import pandas as pd
import numpy as np

In [ ]:
# mount google drive to import csv data file
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
# import the csv data file from google drive
path='/content/drive/MyDrive/_PROJECTS_/Self_EDA/'
imdb_df = pd.read_csv(path+'imdb_data.csv')

In [ ]:
imdb_df.head()

In [ ]:
imdb_df.tail()

In [ ]:
imdb_df.info()

Keep all the non null columns for the answers to the questions

In [ ]:
columns_to_keep= ['imdb_id','budget', 'genres','original_language', 'original_title','cast', 'crew', 'revenue']

In [ ]:
imdb_df['genres']

In [ ]:
# check the first value in genres
imdb_df.loc[0,'genres']

In [ ]:
# check type of first value of genre column
type(imdb_df.loc[0,'genres'])

##Write function to convert string to list

In [ ]:
def convert_to_list(str):
  return eval(str)

In [ ]:
# find all the row index non null values in genre string
imdb_df.loc[~imdb_df['genres'].isna(),'genres']

In [ ]:
#apply the above function only on non null values in genres column
imdb_df.loc[~imdb_df['genres'].isna(),'genres']= imdb_df.loc[~imdb_df['genres'].isna(),'genres'].apply(convert_to_list)

In [ ]:
#apply the above function only on non null values in cast column
imdb_df.loc[~imdb_df['cast'].isna(),'cast']= imdb_df.loc[~imdb_df['cast'].isna(),'cast'].apply(convert_to_list)

In [ ]:
#apply the above function only on non null values in crew column
imdb_df.loc[~imdb_df['crew'].isna(),'crew']= imdb_df.loc[~imdb_df['crew'].isna(),'crew'].apply(convert_to_list)

In [ ]:
#creating a df with the required columns
df = imdb_df[columns_to_keep]

In [ ]:
df.head(2)

In [ ]:
df.info()

Drop the null values in the df

In [ ]:
df.dropna(inplace=True)

In [ ]:
df.info()

In [ ]:
df.describe()

Minimum value of budget and revenue is very low ( 0 and 1) which is not possible so replace very low values(below 1000) with the median of the budget above 1000

In [ ]:
df.loc[df['budget']<1000,'budget'] = df.loc[df['budget']>=1000,'budget'].median()

In [ ]:
df.loc[df['revenue']<1000,'revenue'] = df.loc[df['revenue']>=1000,'revenue'].median()

In [ ]:
df.describe()

In [ ]:
#create profit and ROI column
df['profit'] = df['revenue'] - df['budget']
df['roi']= 100* (df['profit']/df['budget'])

In [ ]:
df.describe()

#Which movie made the highest profit? Who were its producer and director? Identify the actors in that film.

In [ ]:
# max profit
df['profit'].max()

In [ ]:
#find index or row which have the max profit using .idxmax()
#.idxmax()-->> returns the row number(index) for the max value of the column 
df['profit'].idxmax()

In [ ]:
# name of the movie for which the highest profit
df.loc[df['profit'].idxmax(),'original_title']

In [ ]:
# create a max profit movie df
max_profit_movie_df = df.loc[df['profit'].idxmax()]

In [ ]:
max_profit_movie_df.head()

In [ ]:
max_profit_movie_df.loc['cast'][0]['name']

In [ ]:
# create a crew list for max profit movie
crew_list= max_profit_movie_df.loc['crew']

# view first 3 crew
crew_list[0:3]

##Name of the director and producer of the max profit movie

In [ ]:
producer_list=[]
director_list=[]
for elem in crew_list:
  if elem['job']=='Producer':
    producer_list.append(elem['name'])
  if elem['job']=='Director':
    director_list.append(elem['name'])

In [ ]:
print(f'PRODUCERS : {producer_list}')
print(f'DIRECTORS : {director_list}')

In [ ]:
# cast list

cast_list =max_profit_movie_df['cast']
# view first 3 cast
cast_list[0:3]

In [ ]:
# list of actors in highest profit movie
actor_list=[]
for elem in cast_list:
  actor_list.append(elem['name'])

In [ ]:
#actors
print(f'Actors of the movie are :')
actor_list

#This data has information about movies made in different languages. Which language has the highest average ROI (return on investment)?

In [ ]:
# find the roi mean for each language
df.groupby('original_language')['roi'].mean().reset_index()

In [ ]:
roi_languages_df = df.groupby('original_language')['roi'].mean().reset_index()

In [ ]:
print('Language with highest average roi is')
df.groupby('original_language')['roi'].mean().reset_index().sort_values(by='roi',ascending=False).iloc[0,0]

#Find out the unique genres of movies in this dataset.

In [ ]:
no_na_genres = df[~df['genres'].isna()]

In [ ]:
#create a list of genres and using .iterrow() method to iterate over genres column 
# .iterrow() --->> same as enumerate() its compulsory to use it in case of DataFrame
gen_list=[]
for index,row in no_na_genres.iterrows():    #tuple of index and row
  genre = no_na_genres.loc[index,'genres']
  for k in genre:
    gen_list.append(k['name'])
  
#unique list of genres are:
pd.DataFrame(set(gen_list),columns=['Unique Genres'])

#Make a table of all the producers and directors of each movie. Find the top 3 producers who have produced movies with the highest average RoI?

In [ ]:
no_na_crew = df[~df['crew'].isna()]

In [ ]:
no_na_crew.shape

In [ ]:
#A simple function to extract list of all producers for a given movie_index
def create_producer_list(index):
  movie_index = no_na_crew.iloc[index]
  crew_list= movie_index.loc['crew']
  producer_list=[]
  for elem in crew_list:
     if elem['job']=='Producer':
        producer_list.append(elem['name'])
  return producer_list

In [ ]:
create_producer_list(10)

In [ ]:
#A simple function to extract names of all directors for a given movie_index
#each movie has only one director
def create_director(index):
  movie_index = no_na_crew.iloc[index]
  crew_list= movie_index.loc['crew']

  for elem in crew_list:
     if elem['job']=='Director':
        return elem['name']

In [ ]:
create_director(10)

In [ ]:
#create a empty DataFrame with required Column names in which we will append data later
Table = pd.DataFrame(columns=['Movie Title','Producers','Directors','ROI'])

In [ ]:
for index,row in no_na_crew.iterrows():
  
  try:
      Table = Table.append({'Movie Title':no_na_crew.loc[index,'original_title'],'Producers':create_producer_list(index),'Directors':create_director(index),'ROI':no_na_crew.loc[index,'roi']},ignore_index=True)
  except:
    continue
    


In [ ]:
# Table containing columns of Movie Title,its Producers,Directors and ROI

Table.head(10)

#Which actor has acted in the most number of movies? Deep dive into the movies, genres and profits corresponding to this actor.

In [ ]:
no_na_cast = df[~df['cast'].isna()]

In [ ]:
no_na_cast.loc[0,'cast'][0]['name']

In [ ]:
actor_list=[]
for index,row in no_na_cast.iterrows():
  for iter in no_na_cast.loc[index,'cast']:
    if type(iter)== dict:
      actor= iter['name']
      actor_list.append(actor)

In [ ]:
#create a  DataFrame with actor list 
Actor_Table = pd.DataFrame(actor_list,columns=['Name of Actor'])

In [ ]:
Actor_Table.shape

In [ ]:
Actor_Table.head()

In [ ]:
#sorting the actors using groupby function
Actor_Table.value_counts().reset_index().head()

In [ ]:
# Actors who have done maximum movies are(from the above groupby function):
print('Samuel L. Jackson and Robert De Niro both have done 30 films')

In [ ]:

profit1=[]
profit2=[]
movie1=[]
movie2=[]
for index,row in no_na_cast.iterrows():
  for iter in no_na_cast.loc[index,'cast']:
    if type(iter)== dict:
      actor= iter['name']
      if 'Robert De Niro' in actor:
        profit1.append(no_na_cast.loc[index,'profit'])
        movie1.append(no_na_cast.loc[index,'original_title'])
      

        

      if 'Samuel L. Jackson' in actor:
        profit2.append(no_na_cast.loc[index,'profit'])
        movie2.append(no_na_cast.loc[index,'original_title'])
        

In [ ]:
#creating a loop to get the genres for Robert and Samuel
gener_r=[]
a=[]
for i in range(len(movie1)):
  for g in no_na_cast.loc[i,'genres']:
    a.append(g['name'])
    
  gener_r.append(a)
  a=[]

gener_s=[]
b=[]
for i in range(len(movie2)):
  for g in no_na_cast.loc[i,'genres']:
    b.append(g['name'])
    
  gener_s.append(b)
  b=[]



In [ ]:
genr = np.array(gener_r)
gens = np.array(gener_s)

In [ ]:
#creating sub dataframe for Robert
mov1= pd.DataFrame(movie1,columns=['Movie Name'])
prof1=pd.DataFrame(profit1,columns=['Movie Profit'])
gen1= pd.DataFrame(genr.flatten(),columns=['Genres'])

In [ ]:
Movies_by_Robert=pd.concat([mov1,gen1,prof1],axis=1)

In [ ]:
# Movies by Robert De Niro
Movies_by_Robert.sort_values(by='Movie Profit',ascending=False).head()

In [ ]:
#creating sub dataframe for Samuel
mov2= pd.DataFrame(movie2,columns=['Movie Name'])
prof2=pd.DataFrame(profit2,columns=['Movie Profit'])
gen2= pd.DataFrame(gens.flatten(),columns=['Genres'])

In [ ]:
Movies_by_Samuel=pd.concat([mov1,gen1,prof1],axis=1)

In [ ]:
# Movies by Samuel L jackson
Movies_by_Samuel.sort_values(by='Movie Profit',ascending=False).head()

#Conclusion
With the help of pandas library, the exploratory data analysis of the imdb_data.csv is done.
The required data is imported from google drive to dataframe.
The data is explored to evaluate the sanity and all only the required columns are used to create a working dataframe.

All the columns that had string data type for the list of dictionary were converted to the list datatype with the help of custom function and eval function. All the rows with null values in the working dataframe were dropped.

Solution to the questions are obtained by using various functions and methods.